In [1]:
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


print(tf.__version__)
print(np.__version__)

2.3.0
1.18.5


In [2]:
# divide uma série temporal univariada em amostras
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # encontra o final da amostra
        end_ix = i + n_steps
        # verifica se passou do final
        if end_ix > len(sequence)-1:
            break
        # junta as partes de entrada e saída do padrão
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
def train_test_split(data, train_portion):
    time_len = data.shape[1]
    train_size = int(time_len * train_portion)
    train_data = np.array(data.iloc[:, :train_size])
    test_data = np.array(data.iloc[:, train_size:])
    return train_data, test_data

In [4]:
def sequence_data_preparation(seq_len, pre_len, train_data, test_data):
    trainX, trainY, testX, testY = [], [], [], []

    for i in range(train_data.shape[1] - int(seq_len + pre_len - 1)):
        a = train_data[:, i : i + seq_len + pre_len]
        trainX.append(a[:, :seq_len])
        trainY.append(a[:, -1])

    for i in range(test_data.shape[1] - int(seq_len + pre_len - 1)):
        b = test_data[:, i : i + seq_len + pre_len]
        testX.append(b[:, :seq_len])
        testY.append(b[:, -1])

    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)

    return trainX, trainY, testX, testY

In [5]:
df = pd.read_csv('sp_beaches_update.csv', parse_dates=['Date'])
df = df.sort_values(by=['Date'])
df=df.loc[~df['Enterococcus'].isnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74171 entries, 0 to 70635
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   City          74171 non-null  object        
 1   Beach         74171 non-null  object        
 2   Date          74171 non-null  datetime64[ns]
 3   Enterococcus  74171 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.8+ MB


In [6]:
cidade="UBATUBA"
praia="GRANDE"
 
df_beach = df.loc[df['City']==cidade].loc[df['Beach']==praia][['Date','Enterococcus']]
df_beach.columns = ['ds', 'y']
df_beach.set_index('ds', inplace=True)
df_beach

,y
ds,
2012-01-03,5
2012-01-08,42
2012-01-15,92
2012-01-22,16
2012-01-29,9
...,...
2021-07-12,1
2021-07-19,38
2021-07-26,17


In [7]:
train_rate = 0.9
train_data, test_data = train_test_split(df_beach, train_rate)
print("Train data: ", train_data.shape)
print("Test data: ", test_data.shape)

Train data:  (468, 0)
Test data:  (468, 1)


In [8]:
seq_len = 16 #utilizar as ultimas 16 medições
pre_len = 1 #para prever 1 medição
trainX, trainY, testX, testY = sequence_data_preparation(
    seq_len, pre_len, train_data, test_data
)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(0,)
(0,)
(0,)
(0,)
